In [1]:
from google.colab import drive
drive.mount('/content/drive/')
import shutil
from google.colab import files

Mounted at /content/drive/


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.layers import Dense, Flatten, LSTM, Dropout, SimpleRNN
from keras.models import Sequential
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from keras.utils import to_categorical

In [3]:
data_path = "/content/drive/MyDrive/Location_Season_Data/"

In [4]:
df = pd.read_csv(data_path + "Location_Season_Pressure_first_3_year_ROI.csv")
df

,0,1,2,3,4,5,6,7,8,9,...,223,224,225,226,227,228,season,Name,Day,Class
0,0.475692,0.473846,0.471385,0.468923,0.465846,0.462769,0.460308,0.456615,0.453538,0.450462,...,0.435077,0.443077,1,1,1,0,2,116_c7_cp2_100_32,116,0
1,0.464615,0.463385,0.462769,0.460923,0.460308,0.457846,0.455385,0.452923,0.451077,0.450462,...,0.454769,0.475077,1,0,1,0,2,125_c3_cp2_49_32,125,0
2,0.248615,0.243692,0.240000,0.237538,0.235692,0.234462,0.234462,0.235692,0.237538,0.240615,...,0.246154,0.264615,1,0,0,0,2,120_c1_cp2_31_29,120,0
3,0.552000,0.544000,0.536615,0.529846,0.524923,0.521846,0.520000,0.519385,0.520615,0.523077,...,0.475077,0.492923,1,1,1,0,2,127_c3_cp3_96_38,127,0
4,0.248615,0.243692,0.240000,0.237538,0.235692,0.234462,0.234462,0.235692,0.237538,0.240615,...,0.246154,0.264615,1,0,0,0,2,120_c2_cp2_31_29,120,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,0.930833,0.935248,0.939601,0.937135,0.942242,0.940599,0.940394,0.935167,0.934217,0.936547,...,0.948077,0.953185,1,0,1,1,3,997_c12_cp2_62_59,997,3
1064,0.869459,0.873418,0.876279,0.877193,0.879589,0.879725,0.879310,0.874263,0.870889,0.869159,...,0.911058,0.919884,1,0,1,1,3,998_c12_cp2_37_48,998,2
1065,0.759864,0.772639,0.783731,0.791910,0.800783,0.806087,0.810837,0.810413,0.810015,0.810133,...,0.813462,0.813224,1,1,1,0,3,999_c11_cp2_102_34,999,2
1066,0.812957,0.820837,0.826595,0.829435,0.835047,0.838979,0.843350,0.843320,0.844379,0.848008,...,0.909615,0.921332,1,0,1,0,3,999_c11_cp3_37_44,999,2


In [8]:
df_train = df[(df["Day"] >= 0) & (df["Day"] <= 730)]
X_train = df_train.drop(columns = ["Name", "Day", "Class"])
Y_train = df_train["Class"]

df_test = df[(df["Day"] >= 731) & (df["Day"] <= 1095)]
X_test = df_test.drop(columns = ["Name", "Day", "Class"])
Y_test = df_test["Class"]

In [9]:
#Reshape training and testing data
train_X = X_train.to_numpy().reshape(X_train.shape[0],23,10,1).astype("float32")

train_Y = Y_train.to_numpy()

test_X = X_test.to_numpy().reshape(X_test.shape[0],23,10,1).astype("float32")

test_Y = Y_test.to_numpy()

train_Y = to_categorical(train_Y, num_classes=4)
test_Y = to_categorical(test_Y, num_classes=4)

In [30]:
#Define the model architecture
model_rnn = Sequential()
model_rnn.add(LSTM(16, input_shape=(23,10), return_sequences=True))
model_rnn.add(Dropout(0.1))
model_rnn.add(LSTM(16, return_sequences=True))
model_rnn.add(Dropout(0.1))
model_rnn.add(LSTM(16))
model_rnn.add(Dense(4, activation='softmax'))

In [31]:
model_rnn.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [32]:
early_stopping = EarlyStopping(monitor='loss', patience=30)


#Fit the model
model_rnn.fit(train_X, train_Y, epochs = 1000, batch_size = 64, verbose = 2, callbacks = early_stopping)

Epoch 1/1000
12/12 - 6s - loss: 1.3269 - accuracy: 0.4450 - 6s/epoch - 504ms/step
Epoch 2/1000
12/12 - 0s - loss: 1.2724 - accuracy: 0.4573 - 380ms/epoch - 32ms/step
Epoch 3/1000
12/12 - 0s - loss: 1.2372 - accuracy: 0.4925 - 365ms/epoch - 30ms/step
Epoch 4/1000
12/12 - 0s - loss: 1.2038 - accuracy: 0.5319 - 394ms/epoch - 33ms/step
Epoch 5/1000
12/12 - 0s - loss: 1.1481 - accuracy: 0.5577 - 392ms/epoch - 33ms/step
Epoch 6/1000
12/12 - 0s - loss: 1.0547 - accuracy: 0.5509 - 369ms/epoch - 31ms/step
Epoch 7/1000
12/12 - 0s - loss: 0.9856 - accuracy: 0.5617 - 371ms/epoch - 31ms/step
Epoch 8/1000
12/12 - 0s - loss: 0.9746 - accuracy: 0.5360 - 359ms/epoch - 30ms/step
Epoch 9/1000
12/12 - 0s - loss: 0.9386 - accuracy: 0.5984 - 373ms/epoch - 31ms/step
Epoch 10/1000
12/12 - 0s - loss: 0.9148 - accuracy: 0.5753 - 365ms/epoch - 30ms/step
Epoch 11/1000
12/12 - 0s - loss: 0.8888 - accuracy: 0.5997 - 462ms/epoch - 39ms/step
Epoch 12/1000
12/12 - 1s - loss: 0.9008 - accuracy: 0.6024 - 604ms/epoch - 5

In [33]:
# predict class probabilities for test set
Y_pred = model_rnn.predict(test_X)

# convert predicted probabilities to class labels
Y_pred_labels = np.argmax(Y_pred, axis=1)


# convert true labels to class labels
Y_true_labels = np.argmax(test_Y, axis=1)

#Accuracy
accuracy = accuracy_score(Y_true_labels, Y_pred_labels)
print(accuracy)

# compute confusion matrix
conf_matrix = confusion_matrix(Y_true_labels, Y_pred_labels)

print(conf_matrix)

11/11 [==============================] - 2s 9ms/step
0.4561933534743202
[[40  0 49  0]
 [ 0 23  7 49]
 [ 6  0 81  0]
 [ 0  1 68  7]]
